<a href="https://colab.research.google.com/github/Souleymaneawe/Souleymaneawe/blob/main/Plateforme_de_E_Learning_avec_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gradio as gr
import requests
import json
import os


In [6]:
# Configuration de l'API Gemini (remplacez par vos informations d'API réelles)
GEMINI_API_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent"  # URL de l'API Gemini
# Import userdata to securely store your API key
from google.colab import userdata
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")  # Get the API key from Colab's Secrets Manager

# Charger les données des cours à partir du fichier JSON
def load_courses():
    try:
        with open("courses.json", "r") as file:
            return json.load(file)
    except FileNotFoundError:
        return {
            "courses": [
                {"name": "Python pour Débutants", "description": "Apprenez les bases de la programmation Python."},
                {"name": "Mathématiques Avancées", "description": "Explorez les concepts avancés en algèbre et calcul."},
                {"name": "Histoire Moderne", "description": "Découvrez les événements clés du 20e siècle."}
            ]
        }

# Fonction pour appeler l'API Gemini
def get_gemini_response(prompt):
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "contents": [{"parts": [{"text": prompt}]}]
    }
    try:
        response = requests.post(f"{GEMINI_API_URL}?key={GEMINI_API_KEY}", headers=headers, json=data)
        response.raise_for_status()
        result = response.json()
        return result.get("candidates", [{}])[0].get("content", {}).get("parts", [{}])[0].get("text", "Erreur de réponse de l'API")
    except requests.exceptions.RequestException as e:
        return f"Erreur API : {str(e)}"

# Fonction pour la recommandation de contenu
def get_recommendation(course):
    prompt = f"Recommande une leçon pour un étudiant suivant le cours {course}."
    return get_gemini_response(prompt)

# Fonction pour le quiz
def check_quiz_answer(course, question, answer):
    prompt = f"Vérifie si la réponse '{answer}' est correcte pour la question '{question}' dans le cours {course}."
    return get_gemini_response(prompt)

# Interface Gradio
with gr.Blocks(title="Plateforme de E-Learning avec Gemini") as app:
    gr.Markdown("# Plateforme de E-Learning avec Gemini")

    # Charger les cours
    courses_data = load_courses()
    course_names = [course["name"] for course in courses_data["courses"]]

    # Section pour les cours
    gr.Markdown("## Cours Disponibles")
    course = gr.Dropdown(choices=course_names, label="Choisissez un cours")
    course_description = gr.Textbox(label="Description du cours", interactive=False)

    # Mettre à jour la description du cours sélectionné
    def update_course_description(selected_course):
        for course_data in courses_data["courses"]:
            if course_data["name"] == selected_course:
                return course_data["description"]
        return "Cours non trouvé."

    course.change(fn=update_course_description, inputs=course, outputs=course_description)

    # Recommandation de contenu
    gr.Markdown("## Recommandation de Contenu")
    recommend_button = gr.Button("Obtenir une Recommandation")
    recommendation_output = gr.Textbox(label="Recommandation")
    recommend_button.click(fn=get_recommendation, inputs=course, outputs=recommendation_output)

    # Quiz interactif
    gr.Markdown("## Quiz")
    question = gr.Textbox(label="Posez une question")
    answer = gr.Textbox(label="Votre réponse")
    quiz_button = gr.Button("Vérifier la Réponse")
    quiz_output = gr.Textbox(label="Résultat")
    quiz_button.click(fn=check_quiz_answer, inputs=[course, question, answer], outputs=quiz_output)

In [5]:
app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b85dc0bc54e5a1183e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
